# **Gene Feature Extraction from FASTA sequence**

This project involves the development of a command-line Python script designed for gene feature extraction from **FASTA** sequence data. The tool supports multi-sequence FASTA inputs and performs key computations such as sequence length, GC/AT content, and **open reading frame (ORF)** detection. Additionally, it provides an option to visualize ORFs graphically and export results to CSV format upon user prompt. This script serves as a lightweight, modular bioinformatics utility for preliminary sequence analysis, particularly useful in genomic data preprocessing workflows.




### **Install Biopython v-1.85**

This command line python script requires biopython to be preinstalled for the proceedings.  Make sure it enabled.


*  If you face an error of package "**Bio**" not found, install biopython through the below command :



In [ ]:
# pip install biopython

Print precaution errors

In [43]:
print("## This software requires biopython pre-installed for running....⏳")
print("Install Biopython by 'pip install biopython' (If you face an error of package 'Bio' not found)...")

## This software requires biopython pre-installed for running....⏳
Install Biopython by 'pip install biopython' (If you face an error of package 'Bio' not found)...


### **Install Dependencies**

In [31]:
from Bio import SeqIO
from Bio.Seq import Seq
import sys
import re
import csv

### **Print Header of CLI**

In [32]:
print("\n")
print("---------------------------------------------------")
print("\n")
print("Welcome to Gene Feature Extractor Software👨‍🔬")
print("🧬 Gene Feature Extractor Started...\n")
print("---------------------------------------------------")

🧬 Gene Feature Extractor Started...



### **Function for nucleotide content calculation**

In [33]:
def calculate_nucleotide_content(seq):
  length = len(seq)
  content = {
      "A" : round( (seq.count("A") / length) * 100 , 2 ),
      "T" : round( (seq.count("T") / length) * 100 , 2 ),
      "G" : round( (seq.count("G") / length) * 100 , 2 ),
      "C" : round( (seq.count("C") / length) * 100 , 2 ),
      "GC_content": round( ( ( seq.count("G") + seq.count("C")) / length) * 100, 2)
  }
  return content

### **To Find the ORF Region**

REASON FOR ORF<30 :
  1. Biological Justification:

     - A codon = 3 bases, so 30 bases = 10 amino acids.

     - Most functional proteins are longer than 10 amino acids.

     - Very short ORFs (<30 bp) are often:

     - False positives (random ATG...TAA).

     - Non-functional peptides.

     - Unlikely to encode real proteins.


  2. Computational Justification:
      
      - Reduces noise from spurious tiny ORFs.

      - Improves performance and result relevance.

      - It's a customizable threshold — users can adjust if they want to study short peptides

In [34]:
def find_orf(seq, min_orf_length = 30):
  orf = []
  start_codon = "ATG"
  stop_codons = ["TAA", "TAG", "TGA"]

  for frame in range(3):
    i = frame
    while i < len(seq) - 3:
      codon = seq[ i:i + 3]
      if codon == start_codon:
        for j in range(i + 3, len(seq) - 2, 3):
          stop_codon = seq[j : j +3]
          if stop_codon in stop_codons:
            orf_seq = seq[i:j + 3 ]
            if len(orf_seq) >= min_orf_length:
              orf.append({
                  "frame" : frame + 1,
                  "start" : i+1,
                  "end" : j + 3,
                  "length" : j+3 - i,
                  "sequence" : orf_seq
              } )
            i = j +3
            break
      i += 3
  return orf

### **To extract Upstream and downstream region**


In [35]:
def extract_upstream_downstream(seq, start, end, size = 50):
  upstream = seq[ max(0, start - size -1): start-1]
  downstream = seq[end:end + size]
  return upstream, downstream

### **To search promotor region**

In [36]:
def search_promotor(upstream_seq):
  motifs = {
      "-10_box (TATAAT)": re.search("TATAAT", upstream_seq),
      "-35_box (TTGACA)": re.search("TTGACA", upstream_seq),
      "Shine-Dalgarno (AGGAGG)" : re.search("AGGAGG", upstream_seq)
  }

  return {k: ("Found" if v else "Not Available") for k, v in motifs.items()}

### **To display ORF mapping**

In [37]:
def display_orf_map(seq_len, orf):
  bar = [" . "] * seq_len
  for orf in orf:
    for i in range(orf['start'] - 1, orf['end']):
      bar[i] = '#'
  return ''.join(bar[:100]) + ("..." if seq_len > 100 else "")

### **Analysze FASTA input file**

In [41]:
def analyze_fasta(file_path):
  results = []
  for record in SeqIO.parse(file_path, "fasta"):
    seq_id = record.id
    sequence = str(record.seq).upper()
    orfs = find_orf(sequence)
    print(f"\n🧬 Sequence ID: {seq_id} | Length : {len(sequence)} bp | ORFs Found : {len(orfs)}")

    if not orfs:
      print("No ORFs found.")
      continue

    print("ORF Map (first 100 bp shown) : " )
    print(display_orf_map(len(sequence), orfs))

    for idx, orf in enumerate(orfs, 1):
      content = calculate_nucleotide_content(orf['sequence'])
      upstream, downstream = extract_upstream_downstream(sequence, orf['start'], orf['end'])
      motifs = search_promotor(upstream)

      print(f"\n---# ORF {idx} # ---")
      print(f"Start-End         : {orf['start']} - {orf['end']}")
      print(f"Length            : {orf['length']} bp | Frame: {orf['frame']}")
      print(f"ATGC Content %    : A: {content['A']}% | T: {content['T']}% | G: {content['G']}% | C: {content['C']}%")
      print(f"GC Content %      : {content['GC_content']}%")
      print(f"Upstream (50bp)   : {upstream if upstream else 'Not Available'}")
      print(f"Downstream (50bp) : {downstream if downstream else 'Not Available'} ")

      for name, status in motifs.items():
        print(f"{name:25} : {status}")

      results.append({
          "Sequence_ID" : seq_id,
          "ORF_ Number" : idx,
          "Start" : orf['start'],
          "End" : orf['end'],
          "Length" : orf['length'],
          "Frame" : orf['frame'],
          "GC_Content" : content['GC_content'],
          "-10_box (TATAAT)" : motifs["-10_box (TATAAT)"],
          "-35_box (TTGACA)" : motifs["-35_box (TTGACA)"],
          "Shine-Dalgarno (AGGAGG)" : motifs["Shine-Dalgarno (AGGAGG)"]
      })

  return results



### **Save file as CSV**

In [22]:
def prompt_csv(results):
  choice = input("\n💾 Do you want to save the results to a CSV file? (y/n): ").strip().lower()

  if choice == "y":
    outname = "analyze_results.csv"
    keys = results [0].keys()

    with open(outname, "w", newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = keys)
      writer.writeheader()
      writer.writerows(results)

    print(f"✅ Results saved to: {outname}")
    print(f"\nThanks for using our software✨")
    print(f"\n©️Rights owned by KARTHIK YOGANANTHAM")
    print(f"\n⭕Follow on Github : @KARTHIK-YOGANANTHAM")

  else:
    print("❎ Skipped saving CSV.")
    print(f"\nThanks for using our software✨")
    print(f"\n©️Rights owned by KARTHIK YOGANANTHAM")
    print(f"\n⭕Follow on Github : @KARTHIK-YOGANANTHAM")



### **Execute the Input File**

In [21]:
if __name__ == "__main__":
  if len(sys.argv) != 2:
    print("Usage : python gene_feature_extractor.py <input.fasta>")
    sys.exit(1)

  fasta_file = sys.argv[1]
  data = analyze_fasta(fasta_file)
  if data:
    prompt_csv(data)
  else:
    print("⚠️ No ORFs detected in any sequence✨")
    print(f"\nThanks for using our software")
    print(f"\©️Rights owned by KARTHIK YOGANANTHAM")
    print(f"\n⭕Follow on Github : @KARTHIK-YOGANANTHAM")

Usage : python gene_feature_extractor.py <input.fasta>


SystemExit: 1

### **Conclusion**

This Python-based tool was developed to automate the extraction of gene features from DNA sequences provided in a FASTA file. The key focus was on identifying Open Reading Frames (ORFs), exploring their genomic context, and mapping functional regulatory motifs.

🔍 Key Functionalities:
- Multi-sequence support for batch analysis of FASTA files.

- ORF detection based on biological start (ATG) and stop codons (TAA, TAG, TGA).

- Nucleotide composition (%A, T, G, C and GC content) within coding regions.

- Extraction of upstream/downstream sequences for regulatory analysis.

- Motif identification including:

- -10 box (TATAAT)

- -35 box (TTGACA)

- Shine-Dalgarno sequence (AGGAGG)

- ASCII-based ORF visualization to map coding regions within sequences.

- Optional CSV export of annotated results.

### **Contact the developer**

This command line python script was developed by Karthik Yoganantham, a Post graduate biotechnology aspirant willing to combine biological insights with computational resources

For more information Please visit [my website⏏️](https://karthikyoganantham.framer.website/)

- Mail : karthikyoganantham@gmail.com

- Mobile : +91 93841 42469

- Linkedin : https://www.linkedin.com/in/karthik-yoganantham/

- Github : https://github.com/KARTHIK-YOGANANTHAM